# Vehicle License Plate Recognition 之 train model

## 一、構建模型

In [13]:
import cv2
import os
import keras
from keras.models import model_from_json, load_model
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Input
from keras.layers import AveragePooling2D, Flatten, GlobalMaxPooling2D
from keras import layers
from keras.models import Model
import tensorflow as tf
import numpy as np

config = {'黑': 36, 'B': 19, '桂': 45, '2': 62, '陕': 29, '浙': 48, 'N': 3, '1': 58, 'K': 20, 'T': 6, '津': 49, '闽': 44,
          'X': 17, '粤': 47, 'Q': 16, 'V': 15, '琼': 41, '皖': 46, '沪': 32, '冀': 52, '鲁': 50, '贵': 35, '川': 31, '吉': 25,
          '豫': 34, '6': 57, 'L': 21, '5': 63, '晋': 28, '4': 60, 'E': 18, '云': 38, 'S': 7, 'J': 12, 'G': 4, '赣': 30,
          'A': 8, 'D': 14, '湘': 40, '鄂': 51, '0': 55, '蒙': 43, 'Y': 22, '辽': 37, 'U': 2, '3': 61, '9': 54, 'W': 24,
          'Z': 5, 'P': 23, 'F': 9, 'M': 11, '8': 59, '7': 56, 'R': 1, 'H': 10, '青': 27, 'C': 13, '苏': 33, '甘': 42,
          '宁': 26, '京': 53, '渝': 39}
config_ = dict(zip(config.values(), config.keys()))


# print(config_)

def conv_block(input_tensor, bn_axis, filters, phase, name, strides=(1, 1)):
    """
    Conv2D 塊，雙路雙卷積計算
    :param input_tensor:(tensor) 輸入張量
    :param filters:(tuple) 卷積核打包
    :param strides:(int) 卷積步長
    :param BN_axis:(int) 規範化卷積軸
    :return: model
    """
    filters1, filters2, filters3 = filters  # 解包卷積核數量
    Conv_base_name = 'Conv_' + name + '_' + str(phase) + '_phase_'
    BN_base_name = 'BN_' + name + '_' + str(phase) + '_phase_'
    x = Conv2D(
        filters=filters1, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2a'
    )(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2a')(x)
    x = Activation(activation='relu')(x)

    x = Conv2D(
        filters=filters2, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2b'
    )(x)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2b')(x)
    x = Activation(activation='relu')(x)

    x = Conv2D(
        filters=filters3, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2c'
    )(x)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2c')(x)
    x = Activation(activation='relu')(x)

    y = Conv2D(filters3, (1, 1), strides=strides, name=Conv_base_name + '1a')(input_tensor)
    y = BatchNormalization(axis=bn_axis, name=BN_base_name + '1b')(y)

    x = layers.add([x, y])
    a = Activation('relu')(x)

    return a


def identity_block(input_tensor, bn_axis, filters, phase, name, strides=(1, 1)):
    """
    Conv2D 塊，雙路單卷積計算
    :param input_tensor:(tensor) 輸入張量
    :param filters:(tuple) 卷積核打包
    :param strides:(int) 卷積步長
    :param BN_axis:(int) 規範化卷積軸
    :return: model
    """
    filters1, filters2, filters3 = filters  # 解包卷積核數量
    Conv_base_name = 'Conv_' + name + '_' + str(phase) + '_phase_'
    BN_base_name = 'BN_' + name + '_' + str(phase) + '_phase_'
    x = Conv2D(
        filters=filters1, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2a'
    )(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2a')(x)
    x = Activation(activation='relu')(x)

    x = Conv2D(
        filters=filters2, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2b'
    )(x)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2b')(x)
    x = Activation(activation='relu')(x)

    x = Conv2D(
        filters=filters3, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2c'
    )(x)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2c')(x)
    x = Activation(activation='relu')(x)

    x = layers.add([x, input_tensor])
    a = Activation('relu')(x)

    return a


def my_resnet():
    inputs = Input(shape=(1, 24, 48))

    x = Conv2D(
        filters=4, kernel_size=(2, 4), padding='same', name='Conv1', data_format='channels_first')(inputs)
    x = BatchNormalization(axis=1, name='BN_Conv1')(x)
    x = Activation('relu')(x)
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format='channels_first')(x)

    x = conv_block(input_tensor=x, bn_axis=1, filters=(4, 4, 64), phase=2, name='a')
    x = identity_block(input_tensor=x, bn_axis=1, filters=(4, 4, 64), phase=2, name='b')
    x = identity_block(input_tensor=x, bn_axis=1, filters=(4, 4, 64), phase=2, name='c')

    # x = conv_block(input_tensor=x, bn_axis=1, filters=(8, 8, 512), phase=3, name='a')
    # x = identity_block(input_tensor=x, bn_axis=1, filters=(8, 8, 512), phase=3, name='b')
    # x = identity_block(input_tensor=x, bn_axis=1, filters=(8, 8, 512), phase=3, name='c')

    x = AveragePooling2D((2, 2), name='avg_pool')(x)
    x = Flatten()(x)
    x = Dense(64, activation='softmax', name='softmax')(x)
    #     x = GlobalMaxPooling2D()(x)

    model = Model(inputs, x, name='My_Resnet')
    return model


def create_model():
    """返回一個已創建好的 resnet model"""
    model = my_resnet()
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model



def get_tfrecord(filename='train.tfrecords', num=1000):
    num=int(num)
    # 將製作好的 tfrecord 數據集文件讀取出來,並轉換成圖片,以驗證數據是否準確無誤
    print('開始導入數據' + filename)
    filename_queue = tf.train.string_input_producer([filename])  # 讀入數據流
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)  # 返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.string),
                                           'img_val': tf.FixedLenFeature([], tf.string),
                                       })  # 取出包含image和label的feature对象
    image = tf.decode_raw(features['img_val'], tf.uint8)
    label = tf.cast(features['label'], tf.string)

    imgdata = []
    imglabel = []
    with tf.Session() as sess:  # 開始一個對話
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        tem = np.zeros((num, 64))
        for i in range(num):
#             if i % 1000 == 0:
#                 print('已完成導入數據' + filename + str(i) + '個')
            example, l = sess.run([image, label])  # 在會話中取出image和label數據
            data = np.resize(example, [48, 24, 3])
            data = turn_two_color(data)
            tem[i][config[l.decode('utf-8')]] = 1
            imgdata.append([data])
            imglabel.append(tem[0])
        coord.request_stop()
        coord.join(threads)
    print('導入數據' + filename + '完成')
    return imgdata, tem


def turn_two_color(data):
    """
    圖片二值化
    :param data: 圖片
    :return: 二值化後的數據
    """
    import cv2
    grayImage = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)

    a = 0
    for i in grayImage:
        b = 0
        for j in i:
            if j < grayImage.mean():  # 比对均值
                grayImage[a][b] = 0
            else:
                grayImage[a][b] = 255
            b += 1
        a += 1
        del b
    grayImage = np.resize(grayImage, [24, 48])
    return grayImage



In [14]:
def main(batch_size=100, epochs=10):
    model = create_model()
    tr_data, tr_label = get_tfrecord(num=13000)
    va_data, va_label = get_tfrecord('validation.tfrecords', num=1000)
    print('數據導入完成，開始計算')
    tem = model.fit(tr_data, tr_label, batch_size=batch_size, epochs=epochs, )

    # print(tem.history)
    model.evaluate(va_data, va_label, batch_size=batch_size)
    model.save('./resnet_1.h5')
    


# main()


In [15]:
def test_model(num=1000.0,filename='test.tfrecords',modelname='./resnet.h5'):
    ts_images, ts_labels = get_tfrecord(filename,num)

    model = load_model(modelname)
    pre = model.predict(ts_images)
    tr = 0
    for a, b in zip(ts_labels, pre):
        a = np.argmax(a)
        b = np.argmax(b)
        if a == b:
            tr += 1
#         else:
#             print('真確：' + config_[a] + " 預測：" + config_[b])
    ls = tr / num
    print("正確率為：{0}".format(ls))
    del ts_images,ts_labels,model,pre
    return ls

test_model()

開始導入數據test.tfrecords
導入數據test.tfrecords完成
正確率為：0.972


0.972

In [9]:
def train_model():
    model = create_model()
    tr_data, tr_label = get_tfrecord(num=13000)
    va_data, va_label = get_tfrecord('validation.tfrecords', num=1000)
    print('數據導入完成，開始計算')
    tem = model.fit(tr_data, tr_label, batch_size=128, epochs=1, )
    model.save('./resnet.h5')
    test_model(filename='validation.tfrecords',num=1000)
    for i in range(5):
        model = load_model('./resnet.h5')
        model.fit(tr_data,tr_label,batch_size=128,epochs=2)
        model.evaluate(va_data, va_label, batch_size=128)
        model.save('./resnet.h5')
        test_model(filename='validation.tfrecords',num=1000)
        
    test_model()
    

train_model()

開始導入數據train.tfrecords
導入數據train.tfrecords完成
開始導入數據validation.tfrecords
導入數據validation.tfrecords完成
數據導入完成，開始計算
Epoch 1/1
13000/13000 [==============================] - 36s 3ms/step - loss: 1.6935 - acc: 0.6648
開始導入數據validation.tfrecords
導入數據validation.tfrecords完成
正確率為：0.729
Epoch 1/2
13000/13000 [==============================] - 33s 3ms/step - loss: 0.4192 - acc: 0.9100
Epoch 2/2
1000/1000 [==============================] - 3s 3ms/step
開始導入數據validation.tfrecords
導入數據validation.tfrecords完成
正確率為：0.914
Epoch 1/2
13000/13000 [==============================] - 33s 3ms/step - loss: 0.1580 - acc: 0.9668
Epoch 2/2
1000/1000 [==============================] - 3s 3ms/step
開始導入數據validation.tfrecords
導入數據validation.tfrecords完成
正確率為：0.945
Epoch 1/2
13000/13000 [==============================] - 33s 3ms/step - loss: 0.0850 - acc: 0.9832
Epoch 2/2
1000/1000 [==============================] - 3s 3ms/step
開始導入數據validation.tfrecords
導入數據validation.tfrecords完成
正確率為：0.956
Epoch 1/2
13000/13000 [==========

TypeError: 'float' object cannot be interpreted as an integer

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled
